In [ ]:
#installing libraries
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers

In [ ]:
#getting csv file
import pandas as pd
df= pd.read_csv('ProcessedDataForChatBot.csv')

In [ ]:
df.head()

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Your API HuggingFace Token"

In [ ]:
#splitting text for each character
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitted_info=[]
for text in df['info']: 
  splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0,)
  output = splitter.split_text(text)
  splitted_info.append(output)

In [ ]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
!pip install faiss-cpu

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

In [ ]:
llm=HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0.6, "max_length":5000})

In [ ]:
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
#Embeddings splitted texts
from langchain.vectorstores import FAISS 

db=[]
for text in splitted_info:
  output = FAISS.from_texts(text, embeddings)
  db.append(output)

In [ ]:
new_list = splitted_info[349:]
for text in new_list:
  output = FAISS.from_texts(text, embeddings)
  db.append(output)

In [ ]:
print(len(db))

In [ ]:
#saving embeddings as csv because it took hours for above block to execute
#you can skip the above block and directly import Embeddings.csv to run the cell below
Embeddings= pd.DataFrame(db)
Embeddings.to_csv(r'C:\Users\my\Desktop\ChatsOfWesteros\Embeddings.csv', index=False, header=False)

In [46]:
#finding answer from splitted text info of particular character
def answer(index, user_input):
  query = user_input
  database= Embeddings[0][index]
  #doing similarity search with present chunks
  docs = database.similarity_search(query)
  return chain.run(input_documents=docs, question=query)

In [47]:
#Example
answer(0, "who is euan?")

'a wizard and Gryffindor student at Hogwarts School of Witchcraft and Wizardry'

In [48]:
def chatbot_response(user_input):
    #first searching by full names
    for index, row in df.iterrows():
        character_name = row['character'].lower()
        if character_name in user_input.lower():
            res= answer(index, user_input)
            if res.lower() == 'not enough information':
                return "Apologies, my knowledge can only go so far..."
            else:
                return res
    #if character not found then search by first or last name
    for index, row in df.iterrows():
        character_name = row['character'].lower()   
        if character_name.split()[0] in user_input.lower() or character_name.split()[-1] in user_input.lower():
            res= answer(index, user_input)
            if res.lower() == 'not enough information':
                return "Apologies, my knowledge can only go so far..."
            else:
                return res
    return "Apologies, my knowledge can only go so far..."
    

In [49]:
#ChatBot
#type 'exit' to exit chatbot
#type your question referring to character

def chat():
    print("WizChat: Greetings wizard, whose secrets do you want to know?")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("WizChat: Goodbye!")
            break
        response = chatbot_response(user_input)
        print("WizChat:", response)

# Start the chatbot
if __name__ == "__main__":
    chat()

WizChat: Greetings wizard, whose secrets do you want to know?
You: Where was Severus born?
WizChat: Spinner's End
You: What is Severus Snape's house?
WizChat: Spinner's End
You: Which house Severus Snape belongs to?
WizChat: Slytherin
You: When was Snape born?
WizChat: 9 January 1960
You: who Snape loved?
WizChat: Lily
You: Who is Harry Potter's father?
WizChat: James Potter
You: Where Harry Potter was born?
WizChat: Godric's Hollow
You: when was Harry Potter born?
WizChat: 31 July 1980
You: Who was Dumbledore's lover?
WizChat: Gellert Grindelwald
You: What is name of Harry Potter's son?
WizChat: Albus
You: Describe Porpington
WizChat: made sneaking out of the castle and avoiding teachers much easier
You: Where Severus died?
WizChat: Shrieking Shack
You: When Tom Riddle met Dumbledore?
WizChat: eleven
You: exit
WizChat: Goodbye!
